In [1]:
import os
from hydrotools.nwis_client.iv import IVDataService
from hydrotools.gcp_client import gcp
from hydrotools.events.event_detection import decomposition as ev
from hydrotools.gcp_client import utils
import pandas as pd
import numpy as np
from pandas import Grouper
import os
from netCDF4 import Dataset
from datetime import date
import requests
import urllib.request
from datetime import datetime, timedelta
import requests


from bs4 import BeautifulSoup
import datetime as dt
from datetime import date


In [3]:
os.chdir('C:\\Users\\nchin\\UF_Research\\CompCon')

In [149]:
sites = pd.read_csv("locations2.csv")

In [8]:
# matched_sites = pd.read_csv("locations.csv")

In [150]:
sites

,SiteName,NWM_ID,USGS_ID
0,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",16768068,2291500
1,"NORTH BRANCH ESTERO RIVER AT ESTERO, FL",16760406,2291580
2,HOWARD CREEK NEAR SARASOTA FL,16837332,2298760
3,BIG SLOUGH CANAL NEAR MYAKKA CITY FL,16837536,2299410
4,SHELL CREEK AT CIRCLE K GROVES NEAR FT OGDEN FL,16813909,2297600
5,BIG SLOUGH AT WEST PRICE BLVD NEAR NORTH PORT FL,16838064,2299472
6,"MYAKKA RIVER NEAR SR 72 NEAR SARASOTA, FL",16838908,2298830
7,MYAKKA RIVER AT CONTROL NEAR LAUREL FL,16837572,2298880
8,SHELL CREEK NEAR PUNTA GORDA FL,16814857,2298202
9,"CALOOSAHATCHEE RIVER AT S-79, NR.OLGA, FLA",10247723,2292900


In [30]:
# matched_sites.drop([12, 15], inplace = True)

In [6]:
sites['USGS_ID'] = '0' + sites['USGS_ID'].astype(str)

In [7]:
NWM_IDs = sites['NWM_ID'].to_list()
USGS_IDs = sites['USGS_ID'].to_list()

### USGS Downloader 

In [8]:
USGS_IDs

['02291500',
 '02291580',
 '02298760',
 '02299410',
 '02297600',
 '02299472',
 '02298830',
 '02298880',
 '02298202',
 '02292900',
 '02293055',
 '02299861',
 '02295637',
 '02296750']

In [9]:
today = date.today()
print("Today date is: ", today)

Today date is:  2021-07-27


In [19]:
sf_data = IVDataService.get(sites=USGS_IDs, parameterCd = '00060',
                        startDT='2018-09-01', endDT= '2021-07-20') #today)

In [20]:
sf_data.to_csv('USGS_sf_20210726.csv', index = False, header = True)

In [21]:
sf_data

,value_date,variable_name,usgs_site_code,measurement_unit,value,qualifiers,series
0,2018-09-01 00:00:00,streamflow,02291500,ft3/s,336.00,['A'],0
1,2018-09-01 00:15:00,streamflow,02291500,ft3/s,336.00,['A'],0
2,2018-09-01 00:30:00,streamflow,02291500,ft3/s,335.00,['A'],0
3,2018-09-01 00:45:00,streamflow,02291500,ft3/s,335.00,['A'],0
4,2018-09-01 01:00:00,streamflow,02291500,ft3/s,335.00,['A'],0
...,...,...,...,...,...,...,...
1350014,2021-07-19 23:00:00,streamflow,02299861,ft3/s,3.33,['P'],0
1350015,2021-07-19 23:15:00,streamflow,02299861,ft3/s,3.50,['P'],0
1350016,2021-07-19 23:30:00,streamflow,02299861,ft3/s,3.50,['P'],0
1350017,2021-07-19 23:45:00,streamflow,02299861,ft3/s,3.33,['P'],0


### GCP Downloader

In [10]:
NWM_ID_list = NWM_IDs

In [11]:
NWM_ID_list

[16768068,
 16760406,
 16837332,
 16837536,
 16813909,
 16838064,
 16838908,
 16837572,
 16814857,
 10247723,
 10247055,
 16863757,
 16803063,
 16807219]

In [13]:
open_file = Dataset(r"‪C:\Users\nchin\UF_Research\CompCon\test2.nc", "r")

NWM_featureIndex_list = list()
fID = open_file.variables['feature_id']
fID.set_always_mask(False)

for ID in NWM_ID_list:
    featureIndex = np.where(fID[:] == ID)
    NWM_featureIndex_list.append(featureIndex[0][0])

In [27]:
NWM_featureIndex_list

[1724737,
 1724617,
 1726914,
 1726982,
 1726655,
 1727128,
 1727264,
 1726990,
 1726706,
 1115386,
 1115220,
 1727834,
 1725686,
 1725982]

In [14]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [61]:
# ##Run #1
# start_year = 2018
# start_month = 9
# start_day = 17

# end_year = 2021
# end_month = 6
# end_day = 27

In [18]:
start_year = 2021
start_month = 4
start_day = 1

end_year = 2021
end_month = 7
end_day = 17

In [19]:
GCP_dates = [dt.strftime('%Y%m%d%H%M') for dt in 
       datetime_range(datetime(start_year, start_month, start_day), datetime(end_year, end_month, end_day), 
       timedelta(days = 1))]

In [38]:
os.chdir('C:\\Users\\nchin\\UF_Research\\CompCon\\NWM_files')

In [20]:
dts = [dt.strftime('%Y%m%d%H%M') for dt in 
       datetime_range(datetime(start_year, start_month, start_day), datetime(end_year, end_month, end_day), 
       timedelta(hours = 6))]

In [23]:
#because the format changes on the date below, use index changer to change the URL
# idx_changer = dts.index('201906190600')
#wont need unless getting data before date above ^ due to format change

In [24]:
# idx_changer

In [25]:
#forecast hour format "000"
#since model is being run in 3 day intervals 
#use 72 hours for forcasting time
hour_forecast = '072'

In [16]:
# for GCP_date in GCP_dates:
#     print(GCP_date[:8])

In [26]:
for x in range(0, 19, 6):        
    interval_6hr = str(x)     
    print(interval_6hr)

0
6
12
18


In [12]:
# sfList = list()
# dateList = list()
# NWM_IDList = list()
# # USGS_IDList = list()
# #predicted_datetime = list()#add 3 days

# url_switch = 0
# for GCP_date in GCP_dates:
#     for x in range(0, 19, 6):
#         url_switch += 1
#         interval_6hr = str(x)
        
#         if x < 10:
#             interval_6hr = '0'+interval_6hr        
        
#         url = ''
#         if url_switch <= idx_changer: 
#             #download file
#             url = 'https://storage.googleapis.com/national-water-model/nwm.' + GCP_date[:8] +\
#                 '/medium_range/nwm.t' + interval_6hr + 'z.medium_range.channel_rt.f' + hour_forecast + '.conus.nc'
#         else:
#             url = 'https://storage.googleapis.com/national-water-model/nwm.' + GCP_date[:8] +\
#             '/medium_range_mem1/nwm.t' + interval_6hr + 'z.medium_range.channel_rt_1.f' + hour_forecast +'.conus.nc'
        
#         try:
#             filename = GCP_date[:8] + interval_6hr + '00'
#             urllib.request.urlretrieve(url, (filename))

#             #Scrape file
#             open_file = Dataset('C:\\Users\\nchin\\UF_Research\\CompCon\\GCP_downloads\\' + filename, "r")

#             #Create a running list of locations
#             num_locations = len(NWM_featureIndex_list)

#             for i in range(num_locations):            
# #                 USGS_IDList.append(USGS_site_code_list[i]) 
#                 NWM_IDList.append(NWM_ID_list[i])

#                 streamflow_array = open_file.variables['streamflow']
#                 sf = streamflow_array[NWM_featureIndex_list[i]]
#                 sfList.append(sf)

#                 d = str(filename)[:12]
#                 dateList.append(d) 
#             open_file.close()

#             os.remove(filename)
        
#         except urllib.error.HTTPError as exception:
#             continue 

In [39]:
sfList = list()
dateList = list()
NWM_IDList = list()
# USGS_IDList = list()
# predicted_datetime = list()#add 3 days

url_switch = 0
for GCP_date in GCP_dates:
    for x in range(0, 19, 6):        
        interval_6hr = str(x)        
        if x < 10:
            interval_6hr = '0'+interval_6hr        
            
        url = 'https://storage.googleapis.com/national-water-model/nwm.' + GCP_date[:8] +\
        '/medium_range_mem1/nwm.t' + interval_6hr + 'z.medium_range.channel_rt_1.f' + hour_forecast +'.conus.nc'
        
        #print(url)
        try:
            filename = GCP_date[:8] + interval_6hr + '00'
            urllib.request.urlretrieve(url, (filename))

            #Scrape file
            open_file = Dataset('C:\\Users\\nchin\\UF_Research\\CompCon\\NWM_files\\' + filename, "r")

            #Create a running list of locations
            num_locations = len(NWM_featureIndex_list)

            for i in range(num_locations):            
                NWM_IDList.append(NWM_IDs[i])

                streamflow_array = open_file.variables['streamflow']
                sf = streamflow_array[NWM_featureIndex_list[i]]
                sfList.append(sf)

                d = str(filename)[:12]
                dateList.append(d) 
            open_file.close()

            os.remove(filename)
        
        except urllib.error.HTTPError as exception:
            continue 

In [45]:
predicted_dt_3days = list()
# predicted_dt_3days_int = list()
for d in dateList:
    year = int(d[:4])
    month = int(d[4:6])
    day = int(d[6:8])
    hour = int(d[8:10])
    
    prediction_date = (datetime(year, month, day, hour) + timedelta(hours = 72))
    predicted_dt_3days.append(prediction_date)
    
#     reformatted_dt = prediction_date.strftime('%Y%m%d%H%M')
#     predicted_dt_3days_int.append(reformatted_dt)

In [46]:
datetime_list = list()
for d in dateList:
    year = int(d[:4])
    month = int(d[4:6])
    day = int(d[6:8])
    hour = int(d[8:10])
    
    date = datetime(year, month, day, hour)
    datetime_list.append(date)

In [166]:
NWM_df = pd.DataFrame({'NWM_ID': NWM_IDList, 'datetime_int': dateList, 'datetime': datetime_list, 'predicted_flow': sfList})

In [167]:
NWM_df['predicted_dt_3days'] = predicted_dt_3days
NWM_df = NWM_df.merge(sites)

In [139]:
# NWM_df.drop(columns=['SiteName'], inplace = True)

In [187]:
NWM_df.head(4)

,NWM_ID,datetime_int,datetime,predicted_flow,predicted_dt_3days,SiteName,USGS_ID
0,16768068,202104010000,2021-04-01 00:00:00,0.03,2021-04-04 00:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500
1,16768068,202104010600,2021-04-01 06:00:00,0.03,2021-04-04 06:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500
2,16768068,202104011200,2021-04-01 12:00:00,0.03,2021-04-04 12:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500
3,16768068,202104011800,2021-04-01 18:00:00,0.03,2021-04-04 18:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500


In [188]:
NWM_df.drop(columns = ['datetime_int'], inplace = True)

In [189]:
NWM_df.to_csv('NWMdata_20210717.csv', index = False, header = True)

### 7/26
- Combine the new data with the older dataset
- Merge with USGS data

In [41]:
os.chdir('C:\\Users\\nchin\\UF_Research\\CompCon')

In [173]:
NWM_old = pd.read_csv("NWM_data_20210726_OLD.csv")

In [174]:
NWM_old = NWM_old.loc[NWM_old['NWM_ID'].isin(sites['NWM_ID'].to_list())].copy()

In [176]:
NWM_old.drop(columns = ['USGS_ID'], inplace = True)
NWM_old = NWM_old.merge(sites, how = 'left')

In [178]:
NWM_old.drop(columns = ['predicted_dt_3days_int', 'datetime_int'], inplace = True)

In [179]:
NWM_old

,NWM_ID,datetime,predicted_flow,predicted_dt_3days,SiteName,USGS_ID
0,16803063,2018-09-17 00:00:00,113.309997,2018-09-20 00:00:00,"PEACE RIVER AT US 17 AT ZOLFO SPRINGS, FL",2295637
1,16803063,2018-09-17 06:00:00,86.549998,2018-09-20 06:00:00,"PEACE RIVER AT US 17 AT ZOLFO SPRINGS, FL",2295637
2,16803063,2018-09-17 12:00:00,65.039999,2018-09-20 12:00:00,"PEACE RIVER AT US 17 AT ZOLFO SPRINGS, FL",2295637
3,16803063,2018-09-17 18:00:00,77.419998,2018-09-20 18:00:00,"PEACE RIVER AT US 17 AT ZOLFO SPRINGS, FL",2295637
4,16803063,2018-09-18 00:00:00,70.859998,2018-09-21 00:00:00,"PEACE RIVER AT US 17 AT ZOLFO SPRINGS, FL",2295637
...,...,...,...,...,...,...
61989,16838908,2021-06-05 18:00:00,0.000000,2021-06-08 18:00:00,"MYAKKA RIVER NEAR SR 72 NEAR SARASOTA, FL",2298830
61990,16837572,2021-06-05 18:00:00,0.000000,2021-06-08 18:00:00,MYAKKA RIVER AT CONTROL NEAR LAUREL FL,2298880
61991,16814857,2021-06-05 18:00:00,0.000000,2021-06-08 18:00:00,SHELL CREEK NEAR PUNTA GORDA FL,2298202
61992,10247723,2021-06-05 18:00:00,0.180000,2021-06-08 18:00:00,"CALOOSAHATCHEE RIVER AT S-79, NR.OLGA, FLA",2292900


In [190]:
NWM_20210717 = pd.concat([NWM_df,NWM_old])

In [191]:
NWM_20210717.dtypes

NWM_ID                  int64
datetime               object
predicted_flow        float64
predicted_dt_3days     object
SiteName               object
USGS_ID                 int64
dtype: object

In [104]:
# NWM_20210717['predicted_dt_3days'] = pd.to_datetime(NWM_20210717['predicted_dt_3days'])
# NWM_20210717['datetime'] = pd.to_datetime(NWM_20210717['datetime'])

In [127]:
# NWM_20210717.sort_values(by = 'predicted_dt_3days')

In [192]:
NWM_20210717.drop_duplicates(inplace = True)

In [194]:
NWM_20210717.loc[NWM_20210717.duplicated()]

,NWM_ID,datetime,predicted_flow,predicted_dt_3days,SiteName,USGS_ID


In [199]:
NWM_20210717.reset_index(drop = True, inplace = True)

In [249]:
NWM_20210717.sort_values(by = 'predicted_dt_3days').tail(15)

,NWM_ID,datetime,predicted_flow,predicted_dt_3days,SiteName,USGS_ID
1282,16837332,2021-07-16 12:00:00,11.28,2021-07-19 12:00:00,HOWARD CREEK NEAR SARASOTA FL,2298760
5991,16807219,2021-07-16 18:00:00,0.00,2021-07-19 18:00:00,"PEACE RIVER AT SR 70 AT ARCADIA, FL",2296750
1711,16837536,2021-07-16 18:00:00,0.21,2021-07-19 18:00:00,BIG SLOUGH CANAL NEAR MYAKKA CITY FL,2299410
855,16760406,2021-07-16 18:00:00,0.01,2021-07-19 18:00:00,"NORTH BRANCH ESTERO RIVER AT ESTERO, FL",2291580
2995,16838908,2021-07-16 18:00:00,0.00,2021-07-19 18:00:00,"MYAKKA RIVER NEAR SR 72 NEAR SARASOTA, FL",2298830
5135,16863757,2021-07-16 18:00:00,8.46,2021-07-19 18:00:00,WALKER CREEK NEAR SARASOTA FL,2299861
4279,10247723,2021-07-16 18:00:00,0.46,2021-07-19 18:00:00,"CALOOSAHATCHEE RIVER AT S-79, NR.OLGA, FLA",2292900
2567,16838064,2021-07-16 18:00:00,0.00,2021-07-19 18:00:00,BIG SLOUGH AT WEST PRICE BLVD NEAR NORTH PORT FL,2299472
427,16768068,2021-07-16 18:00:00,0.15,2021-07-19 18:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500
2139,16813909,2021-07-16 18:00:00,0.36,2021-07-19 18:00:00,SHELL CREEK AT CIRCLE K GROVES NEAR FT OGDEN FL,2297600


In [200]:
USGS_data = pd.read_csv("USGS_sf_20210726.csv")

In [221]:
USGS_data['usgs_site_code'].unique()

array([2291500, 2291580, 2292900, 2293055, 2295637, 2296750, 2297600,
       2298202, 2298760, 2298830, 2298880, 2299410, 2299472, 2299861],
      dtype=int64)

In [203]:
def USGS_alter(df):
    dfCopy = df.copy()
    dfCopy['value_date'] = pd.to_datetime(dfCopy['value_date'])
    
    code_changer = {"['A']" : "A",  "['P']": "P",  "['A', 'e']" : "Ae" , 
               "['P', 'Bkw']" : 'P_Bkw', "['P', 'e']": 'Pe'}  
    dfCopy.replace({'qualifiers': code_changer}, inplace = True)    
    dfCopy = dfCopy.loc[(dfCopy['qualifiers'] != "P_Bkw") & (dfCopy['qualifiers'] != "Pe")].copy()
    
    code_int = {"A":3, "Ae":2, "P":1 }
    dfCopy['qualifiers'].replace(code_int, inplace = True)
    
    #Rolling Centered Moving Average
    dfCopy.set_index('value_date', inplace = True)
    dfCopy = dfCopy.groupby('usgs_site_code').rolling(24)['value', 'qualifiers'].mean().reset_index()
#     return dfCopy
    dfCopy.set_index('value_date', inplace = True)
    dfCopy.index = pd.to_datetime(dfCopy.index)
    
    conditions = [
    (dfCopy['qualifiers'] < 1.5),
    (dfCopy['qualifiers'] >= 1.5) & (dfCopy['qualifiers'] < 2.5),
    (dfCopy['qualifiers'] >= 2.5)
    ]
    values = ['P', 'Ae', 'A']
    
    dfCopy['qualifiers'] = np.select(conditions, values)
    
    new_df = pd.DataFrame()
    hrs_of_interest = ['03', '09', '15', '21']
    for hr in hrs_of_interest:
        new_df  = pd.concat([new_df, dfCopy.iloc[dfCopy.index.indexer_at_time(hr + ':00:00')]])
        
    new_df = new_df[new_df['value'].notnull()]    
    
    new_df['centered_6h_MA'] = new_df.index.to_pydatetime() - timedelta(hours = 3)
    
    #ft3/s to m3/s
    new_df['observed_flow'] =  round((new_df['value']*0.028316846592), 4)
    
    return new_df

In [229]:
USGS_updated = USGS_alter(USGS_data)

In [230]:
USGS_updated.dtypes

usgs_site_code             int64
value                    float64
qualifiers                object
centered_6h_MA    datetime64[ns]
observed_flow            float64
dtype: object

In [226]:
USGS_updated.loc[USGS_updated['observed_flow'].isnull()]

,usgs_site_code,value,qualifiers,centered_6h_MA,observed_flow
value_date,,,,,


In [232]:
# USGS_updated['centered_6h_MA'] = USGS_updated['centered_6h_MA'].astype(object)

In [247]:
USGS_updated.sort_values(by = 'centered_6h_MA').tail(17)

,usgs_site_code,value,qualifiers,centered_6h_MA,observed_flow
value_date,,,,,
2021-07-19 15:00:00,2298880,1139.166667,P,2021-07-19 12:00:00,32.2576
2021-07-19 15:00:00,2292900,1842.083333,P,2021-07-19 12:00:00,52.1620
2021-07-19 15:00:00,2291500,73.150000,P,2021-07-19 12:00:00,2.0714
2021-07-19 21:00:00,2298760,8.362917,P,2021-07-19 18:00:00,0.2368
2021-07-19 21:00:00,2299472,133.250000,P,2021-07-19 18:00:00,3.7732
2021-07-19 21:00:00,2298202,371.208333,P,2021-07-19 18:00:00,10.5114
2021-07-19 21:00:00,2298830,894.250000,P,2021-07-19 18:00:00,25.3223
2021-07-19 21:00:00,2297600,94.183333,P,2021-07-19 18:00:00,2.6670
2021-07-19 21:00:00,2299410,62.750000,P,2021-07-19 18:00:00,1.7769


In [ ]:
# NWM_old['USGS_ID'] = '0' + NWM_old['USGS_ID'].astype(str)

In [233]:
NWM_20210717.dtypes

NWM_ID                  int64
datetime               object
predicted_flow        float64
predicted_dt_3days     object
SiteName               object
USGS_ID                 int64
dtype: object

In [234]:
# NWM_20210717 = NWM_20210717.drop(columns=['datetime_int'])
NWM_20210717['predicted_dt_3days'] = pd.to_datetime(NWM_20210717['predicted_dt_3days'])
NWM_20210717['datetime'] = pd.to_datetime(NWM_20210717['datetime'])
NWM_USGS = NWM_20210717.merge(USGS_updated[['usgs_site_code', 'qualifiers', 
            'centered_6h_MA', 'observed_flow']],
            how = 'left', left_on=['USGS_ID', 'predicted_dt_3days'],
            right_on=['usgs_site_code', 'centered_6h_MA'])

In [240]:
null_locations = NWM_USGS.loc[NWM_USGS['observed_flow'].isnull()].index
NWM_USGS.drop(null_locations, inplace = True)
NWM_USGS.drop(columns = ['usgs_site_code', 'centered_6h_MA'], inplace = True)
NWM_USGS.rename(columns = {'predicted_dt_3days': 'PredictionDate_3days'}, inplace = True)
# Peace_matches['USGS_ID'] = '0' + Peace_matches['USGS_ID'].astype(str)

In [241]:
NWM_USGS

,NWM_ID,datetime,predicted_flow,PredictionDate_3days,SiteName,USGS_ID,qualifiers,observed_flow
0,16768068,2021-04-01 00:00:00,0.03,2021-04-04 00:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500,A,0.3289
1,16768068,2021-04-01 06:00:00,0.03,2021-04-04 06:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500,A,0.3313
2,16768068,2021-04-01 12:00:00,0.03,2021-04-04 12:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500,A,0.3313
3,16768068,2021-04-01 18:00:00,0.03,2021-04-04 18:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500,A,0.3247
4,16768068,2021-04-02 00:00:00,0.03,2021-04-05 00:00:00,"IMPERIAL RIVER NEAR BONITA SPRINGS, FL",2291500,A,0.3273
...,...,...,...,...,...,...,...,...
61405,16838908,2021-06-05 18:00:00,0.00,2021-06-08 18:00:00,"MYAKKA RIVER NEAR SR 72 NEAR SARASOTA, FL",2298830,P,0.3733
61406,16837572,2021-06-05 18:00:00,0.00,2021-06-08 18:00:00,MYAKKA RIVER AT CONTROL NEAR LAUREL FL,2298880,P,0.1225
61407,16814857,2021-06-05 18:00:00,0.00,2021-06-08 18:00:00,SHELL CREEK NEAR PUNTA GORDA FL,2298202,P,0.8027
61408,10247723,2021-06-05 18:00:00,0.18,2021-06-08 18:00:00,"CALOOSAHATCHEE RIVER AT S-79, NR.OLGA, FLA",2292900,P,28.8501


In [242]:
NWM_USGS.reset_index(drop = True, inplace=True)

In [243]:
NWM_USGS.to_csv('Streamflow_20210717.csv', index = False, header = True)